In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Dropout # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore

In [4]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../input/train.csv', index_col='id')
X = df.drop(columns=['Tm'])
X = X.drop(columns=['SMILES'])
y = df['Tm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=5, objective='reg:squarederror')
mlp = Sequential()
mlp.add(Dense(units=256, activation='relu', input_shape=(X_train.shape[1],)))
mlp.add(Dropout(0.2))
mlp.add(Dense(units=128, activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(units=64, activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(units=32, activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(units=1))
mlp.compile(loss='mse', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

xgb_oof_preds = np.zeros(X_train.shape[0])
mlp_oof_preds = np.zeros(X_train.shape[0])
meta_y_train = np.zeros(X_train.shape[0]) 

for train_idx, valid_idx in kf.split(X_train):
    
    X_train_fold, X_valid_fold = X_train[train_idx], X_train[valid_idx]
    y_train_fold, y_valid_fold = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    xgb_model.fit(X_train_fold, y_train_fold)
    xgb_oof_preds[valid_idx] = xgb_model.predict(X_valid_fold)
    
    mlp.fit(X_train_fold, y_train_fold, batch_size=128, epochs=100, verbose=1)
    mlp_oof_preds[valid_idx] = mlp.predict(X_valid_fold).flatten()
    
    meta_y_train[valid_idx] = y_valid_fold

stacked_features = np.column_stack((xgb_oof_preds, mlp_oof_preds))

meta_model = LinearRegression()
meta_model.fit(stacked_features, meta_y_train)

xgb_test_preds = xgb_model.predict(X_test)
mlp_test_preds = mlp.predict(X_test)
stacked_test_features = np.column_stack((xgb_test_preds, mlp_test_preds))

final_preds = meta_model.predict(stacked_test_features)

mae_stacked = mean_absolute_error(y_test, final_preds)
print(f"Mean Squared Error (Stacking): {mae_stacked}")


c:\Users\maxjj\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 82778.5234
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 68646.5078
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29290.7617
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 13724.3662
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9579.1807
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7208.0791
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6323.7959
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5994.2646
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5481.7178
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5571.5186
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5019.5332
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 5289.6958
Epoch 13/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5184.9170
Epoch 14/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4917.9473
Epoch 15/100
14/14 ━━━━━